# DyNetworkX Tutorial – Plotting Dynamic Betweenness Centrality

The objective of this tutorial is to showcase a typical use case of DyNetworkX.  
We will be using the Enron dataset, a preview of the data file can be seen below.  
Enron dataset will be downloaded to current directory if not already present. (Size: 2 MB)

In [ ]:
import  os
import urllib
import datetime

file_path = "execs.email.linesnum"

if not os.path.exists(file_path):
    print("Downloading Enron dataset from http://www.cis.jhu.edu/~parky/Enron/execs.email.linesnum ...")
    urllib.request.urlretrieve("http://www.cis.jhu.edu/~parky/Enron/execs.email.linesnum", file_path)
    print("Download complete.")
    
lines = []
with open(file_path, "r") as file:
    for line in file:
        t, u, v = line.split(" ")
        if int(t) > 315522000:
            lines.append(line)
            
with open(file_path, "w") as file:
    for line in lines:
        file.write(line)

### Loading Data into DyNetworkX

Loading data a text file using the function `dnx.ImpulseGraph.load_from_txt`.
Make sure to specify necessary arguments such as `delimiter`, `timestamptype`, and `order`.

Comparing the output of the new ImpulseGraph, it is possible to verify the data set is correctly imported. (Note: order not guaranteed)

In [ ]:
import dynetworkx as dnx

impulseG = dnx.ImpulseGraph.load_from_txt("execs.email.linesnum", delimiter=" ",
                                          timestamptype=int, order=('t', 'v', 'u'))

print(impulseG.edges()[:5])

### Converting between different graph types

Traditionally working with dynamic networks, it is commmon to flatten the temporal dimension by binning data into smaller static graphs called snapshots. This behavior is replicated by the DynetworkX class `SnapshotGraph`.  
By using the argument `length_of_snapshots`, it is possible to specify the desired length of each snapshot to 1 year (converted to seconds to match the data set).

In [ ]:
snapshots = impulseG.to_snapshots(length_of_snapshots=31536000)
first_timestamp = impulseG.interval()[0]
snapshotG = dnx.SnapshotGraph()

i = 0
for snapshot in snapshots:
    snapshotG.add_snapshot(graph=snapshot, start=31536000*i + first_timestamp,
                           end=31536000*(i+1) + first_timestamp)
    i += 1

### Calculating Dynamic Betweenness Centrality

`compute_network_statistic` returns a list, each item in the list refers to each snapshot in the SnapshotGraph. The specified method is applied to each snapshot in the graph, passing additional arguments if present.
The first snapshot can be seen below.

In [ ]:
from networkx.algorithms.centrality import betweenness_centrality

centrality_list = snapshotG.compute_network_statistic(betweenness_centrality)

centrality_sorted = []
unique_nodes = set()
snapshot_list = snapshotG.get()
for snapshot in snapshot_list:
    for node in snapshot.nodes():
        unique_nodes.add(node)

for snapshot in centrality_list:
    centrality_sorted.append(sorted(snapshot.items(), key=lambda x: x[1], reverse=True)+
                             [(float("NaN"), float("NaN"))]*(len(unique_nodes)-len(snapshot)))

print("{:10s}{:10s}{:10s}{:10s}{:10s}{:10s}{:10s}{:10s}".format(
    "Node", "Year 1", "Node", "Year 2", "Node", "Year 3", "Node", "Year 4"))

for i in range(len(unique_nodes)):
    print("{:<10.0f}{:0.3f}\
     {:<10.0f}{:0.3f}\
     {:<10.0f}{:0.3f}\
     {:<10.0f}{:0.3f}".format(
        *centrality_sorted[0][i], *centrality_sorted[1][i],
        *centrality_sorted[2][i], *centrality_sorted[3][i]))

### Formatting Data

From this point forward, we are done using DyNetworkX.
We will finish the objective by getting ready our data to plot.

In [ ]:
min_max = []
for i in range(len(centrality_list)):
    min_value = float("inf")
    max_value = float("-inf")
    for value in centrality_list[i].values():
        min_value = min(min_value, value)
        max_value = max(max_value, value)
    min_max.append((min_value, max_value))
        
snapshot_list = snapshotG.get()

colors = []

for i in range(len(snapshot_list)):
    colors.append([])
    for node in snapshot_list[i]:
        color_value = 0.75*(centrality_list[i][node] - min_max[i][0]) / (min_max[i][1] - min_max[i][0])
        colors[i].append((1-color_value, 1-color_value, 1))

### Plotting Dynamic Betweenness Centrality

Finally, plot the betweenness centrality over time using Matplotlib.animation.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import animation

def update(i, colors, pos, snapshot_list, ax):
    ax.clear()
    ax.set_xlim([-1.1, 1.1])
    ax.set_ylim([-1.1, 1.1])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

    nx.draw_networkx(snapshot_list[i], pos=pos, font_size=16, node_size=900, edgecolors="#000000",
                     node_color=colors[i], ax=ax)

    ax.set_title(f"Year {i+1}", fontsize=24)

fig, ax = plt.subplots(figsize=(20, 20))

staticG = nx.Graph()
for snapshot in snapshotG:
    for u, v in snapshot.edges():
        staticG.add_edge(u, v)
pos = nx.spring_layout(staticG, k=1)

anim = animation.FuncAnimation(fig, update, frames=len(snapshot_list), interval=5000,
                              fargs=(colors, pos, snapshot_list, ax))
anim.save('betweenness.gif', writer='pillow')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

for node in [169, 125, 82, 105]:
    centrality = [0]*4
    for i in range(4):
        if node in centrality_list[i]:
            centrality[i] = centrality_list[i][node]
    plt.plot(range(4), centrality, label=node)

plt.xticks(ticks=range(4), labels=[1, 2, 3, 4])
plt.ylabel("Betweenness Centrality", fontsize=16)
plt.xlabel("Year", fontsize=16)
plt.legend()
plt.show()